# Intro

## Standard modules

In [46]:
import os, sys, pickle
import pathlib
import requests

In [2]:
import numpy as np

In [3]:
from tqdm.auto import tqdm, trange

In [4]:
from sentence_transformers import util

In [5]:
import matplotlib.pyplot as plt
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 2

plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1

In [6]:
from sklearn.decomposition import PCA

In [7]:
import seaborn

In [8]:
import pymannkendall as mk

In [9]:
from scipy.spatial.distance import cosine

In [10]:
import requests
import re
from bs4 import BeautifulSoup  # Per pulizia HTML nelle REST API

In [11]:
import datetime as dt

## Personal modules

In [12]:
from toolbox.updater import time_is_now

In [13]:
from file2text import file2text

In [14]:
from first_rough_cleaning import first_text_cleaner

In [15]:
from file_handler import *

In [16]:
from jina4lote import jina4lote

/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


## Folders

In [17]:
VECTOR_FOLDER='./NewProcessedData/vectors/'

In [18]:
BENCHMARK_FOLDER='./NewProcessedData/benchmarks/'

In [19]:
PCA_CENTR_VECTOR_FOLDER='./NewProcessedData/pca_centr_vectors/'

## Load vectors

In [20]:
vecfiles=os.listdir(PCA_CENTR_VECTOR_FOLDER)

In [21]:
vecfiles.sort()

### Reports' vectors

In [22]:
reports_files=[vecfile for vecfile in vecfiles if vecfile[:4].isnumeric()]

In [23]:
reports_files.sort()

In [24]:
years, k_y=np.unique([rf[:4] for rf in reports_files], return_counts=True)

In [25]:
np.vstack((years, k_y)).T

array([['2015', '40'],
       ['2016', '44'],
       ['2017', '51'],
       ['2018', '56'],
       ['2019', '67'],
       ['2020', '69'],
       ['2021', '76'],
       ['2022', '80'],
       ['2023', '90']], dtype='<U21')

In [26]:
vectors=np.zeros(len(reports_files), dtype=[('year', 'i4'), ('sector', 'U2'), ('company', 'U100'), ('vector', object)])

In [27]:
for i_rf, reports_file in enumerate(reports_files):
    # info from the filename
    splitted_filename=reports_file.split('_')
    
    year=int(splitted_filename[0])
    sector=splitted_filename[1]
    if splitted_filename[-1]=='PLC.txt':
        company='_'.join(splitted_filename[2:-1])
    else:
        company='_'.join(splitted_filename[2:])
        company=company.replace('.txt', '')
    company=company.lower()
    # get the vector
    vector=np.genfromtxt(PCA_CENTR_VECTOR_FOLDER+reports_file)
    
    # organize data
    vectors[i_rf]['year']=year
    vectors[i_rf]['sector']=sector
    vectors[i_rf]['company']=company
    vectors[i_rf]['vector']=vector

### Witten's vector

In [28]:
wittenvecfile=[file for file in os.listdir(PCA_CENTR_VECTOR_FOLDER) if file.startswith('Witten') and file.endswith('.txt')][0]

In [29]:
wittenvec=np.genfromtxt(PCA_CENTR_VECTOR_FOLDER+wittenvecfile)

### Goals' vectors

In [30]:
goalvecfiles=[vecfile for vecfile in vecfiles if vecfile.startswith('Goal')]

In [31]:
goalvecfiles.sort()

In [32]:
goalvecfiles

['Goal-01-Fast-Facts.txt',
 'Goal-02-Fast-Facts.txt',
 'Goal-03-Fast-Facts.txt',
 'Goal-04-Fast-Facts.txt',
 'Goal-05-Fast-Facts.txt',
 'Goal-06-Fast-Facts.txt',
 'Goal-07-Fast-Facts.txt',
 'Goal-08-Fast-Facts.txt',
 'Goal-09-Fast-Facts.txt',
 'Goal-10-Fast-Facts.txt',
 'Goal-11_Fast-Facts.txt',
 'Goal-12_Fast-Facts.txt',
 'Goal-13_Fast-Facts.txt',
 'Goal-14_Fast-Facts.txt',
 'Goal-15-Fast-Facts.txt',
 'Goal-16-Fast-Facts.txt',
 'Goal-17-Fast-Facts.txt',
 'Goal-Fast-Facts.txt']

In [33]:
goalvecs=[np.genfromtxt(PCA_CENTR_VECTOR_FOLDER+goalvecfile) for goalvecfile in goalvecfiles]

### Wikipedia's vectors

In [34]:
wikivecfiles=[vecfile for vecfile in vecfiles if vecfile.startswith('wiki')]

In [35]:
wikivecfiles.sort()

In [36]:
len(wikivecfiles)

4001

In [37]:
wikivecs=[np.genfromtxt(PCA_CENTR_VECTOR_FOLDER+wikivecfile) for wikivecfile in wikivecfiles]

## Sector helper

In [38]:
rtf_files=os.listdir('./rtf')
rtf_files.sort()

In [39]:
sec_helper={}
for file in rtf_files:
    splitted_name=file.split('_')
    if splitted_name[0].isnumeric():
        if splitted_name[0] not in sec_helper.keys():
            sec_helper[splitted_name[0].zfill(2)]=splitted_name[1]

In [40]:
sec_helper

{'10': 'Telecommunications',
 '11': 'Utilities',
 '01': 'Basic Materials',
 '02': 'Consumer Discretionary',
 '03': 'Consumer Staples',
 '04': 'Energy',
 '05': 'Financials',
 '06': 'Health Care',
 '07': 'Industrials',
 '08': 'Real Estate',
 '09': 'Technology'}

## SDG helper

In [41]:
sdg_helper={'01':'No Poverty', 
 '02': 'Zero hunger', 
 '03': 'Good health and well-being', 
 '04': 'Quality education',
 '05': 'Gender equality',
 '06': 'Clean water and sanitation',
 '07': 'Affordable and clean energy',
 '08': 'Decent work and economic growth',
 '09': 'Industry, Innovation, Technology and Infrastructure',
 '10': 'Reduced inequality',
 '11': 'Sustainable cities and communities',
 '12': 'Responsible consumption and production',
 '13': 'Climate action',
 '14': 'Life below water',
 '15': 'Life on land',
 '16': 'Peace, justice and strong institutions',
 '17': 'Partnerships for the goals'
}

## FDR

In [42]:
def fdr_th(p_vals, alpha):
    _aux=np.sort(p_vals)
    fdr_ths=np.arange(1, 1+len(_aux))*alpha/len(_aux)
    fdr_mask=_aux<=fdr_ths
    if np.sum(fdr_mask)>0:
        return fdr_ths[fdr_mask][-1]
    else:
        return 0.

# Validation

In [43]:
years=years.astype('i4')

In [44]:
alpha=0.05

## The benchmark

### Creating the benchmark _[DO NOT EVALUATE IT AGAIN!!!]_

In [48]:
wiki_bench=np.zeros(len(vectors), dtype=[('cos_sim', object)])

In [49]:
for i_v in trange(len(vectors)):
    wiki_bench[i_v]['cos_sim']=np.zeros(len(wikivecs))
    for i_w, wikivec in enumerate(tqdm(wikivecs, leave=False)):
        wiki_bench[i_v]['cos_sim'][i_w]=float(util.cos_sim(wikivec, vectors[i_v]['vector']))

  0%|          | 0/573 [00:00<?, ?it/s]

  0%|          | 0/4001 [00:00<?, ?it/s]

  0%|          | 0/4001 [00:00<?, ?it/s]

  0%|          | 0/4001 [00:00<?, ?it/s]

  0%|          | 0/4001 [00:00<?, ?it/s]

...

  0%|          | 0/4001 [00:00<?, ?it/s]

  0%|          | 0/4001 [00:00<?, ?it/s]

  0%|          | 0/4001 [00:00<?, ?it/s]

  0%|          | 0/4001 [00:00<?, ?it/s]

  0%|          | 0/4001 [00:00<?, ?it/s]

In [51]:
with open('./NewProcessedData/wiki_benchmark.pickle', 'wb') as f:
    pickle.dump(wiki_bench, f)

## All SDGs

In [66]:
with open('./NewProcessedData/wiki_benchmark.pickle', 'rb') as f:
    wiki_bench=pickle.load(f)

In [67]:
cos_sims_global=np.zeros(len(years), dtype=[('mean', 'f8'), ('all', object), ('p_val', object)])

In [68]:
for i_y, year in enumerate(tqdm(years)):
    vecs_y=vectors[vectors['year']==year]['vector']
    wiki_bench_y=wiki_bench[vectors['year']==year]['cos_sim']
    cos_sims_global[i_y]['all']=np.zeros(len(vecs_y))
    cos_sims_global[i_y]['p_val']=np.zeros(len(vecs_y))
    for i_vec, vec in enumerate(tqdm(vecs_y, leave=False)):
        _cos_sim=float(util.cos_sim(goalvecs[-1], vec))
        cos_sims_global[i_y]['all'][i_vec]=_cos_sim
        cos_sims_global[i_y]['p_val'][i_vec]=np.sum(wiki_bench_y[i_vec]>=_cos_sim)/len(wikivecs)
    cos_sims_global[i_y]['mean']=np.mean(cos_sims_global[i_y]['all'])

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

In [69]:
for i_y, year in enumerate(tqdm(years)):
    _fdr_th=fdr_th(cos_sims_global[i_y]['p_val'], alpha)
    compa_y=vectors[vectors['year']==year]['company']
    sec_y=vectors[vectors['year']==year]['sector']
    validated_compa=compa_y[cos_sims_global[i_y]['p_val']<=_fdr_th]
    validated_sec=sec_y[cos_sims_global[i_y]['p_val']<=_fdr_th]
    print(f'{year:}) len(validated_companies)={len(validated_compa):}')
    if len(validated_compa)>0:
        for i_vc, vc in enumerate(validated_compa):
            print(f'\t\t{vc:25} sector={sec_helper[validated_sec[i_vc]]:}')

  0%|          | 0/9 [00:00<?, ?it/s]

2015) len(validated_companies)=4
		unilever                  sector=Consumer Staples
		standard_chartered        sector=Financials
		halma                     sector=Industrials
		land_securities_group     sector=Real Estate
2016) len(validated_companies)=3
		unilever                  sector=Consumer Staples
		standard_chartered        sector=Financials
		halma                     sector=Industrials
2017) len(validated_companies)=1
		halma                     sector=Industrials
2018) len(validated_companies)=2
		standard_chartered        sector=Financials
		halma                     sector=Industrials
2019) len(validated_companies)=0
2020) len(validated_companies)=4
		anglo_american            sector=Basic Materials
		barratt_developments      sector=Consumer Discretionary
		hsbc_holdings             sector=Financials
		standard_chartered        sector=Financials
2021) len(validated_companies)=2
		standard_chartered        sector=Financials
		renewi                    sector=Utilities


## Clustered SDGs

### Define cSDGs

In [70]:
social_sdgs=['01','02','03','04','05','10','16','17']
economic_sdgs=['07','08','09','11','12']
environmental_sdgs=['06','13','14','15']

In [71]:
social_vec=[np.genfromtxt(PCA_CENTR_VECTOR_FOLDER+goalvecfile) for goalvecfile in goalvecfiles if any([sdg in goalvecfile for sdg in social_sdgs])]
social_vec=np.mean(social_vec, axis=0)

In [72]:
economic_vec=[np.genfromtxt(PCA_CENTR_VECTOR_FOLDER+goalvecfile) for goalvecfile in goalvecfiles if any([sdg in goalvecfile for sdg in economic_sdgs])]
economic_vec=np.mean(economic_vec, axis=0)

In [73]:
environmental_vec=[np.genfromtxt(PCA_CENTR_VECTOR_FOLDER+goalvecfile) for goalvecfile in goalvecfiles if any([sdg in goalvecfile for sdg in environmental_sdgs])]
environmental_vec=np.mean(environmental_vec, axis=0)

In [74]:
clustered_sdgs=np.array([social_vec, economic_vec, environmental_vec])

In [75]:
clustered_sdgs.shape

(3, 768)

In [76]:
cSDGS_names=['social', 'economic', 'environmental']

In [77]:
l_csdgs=len(cSDGS_names)

### Cosine similarity validation

In [78]:
cos_sims_cSDGs=np.zeros(len(years)*len(cSDGS_names), dtype=[('cSDG', 'U20'), ('mean', 'f8'), ('all', object), ('p_val', object)])

In [79]:
for i_y, year in enumerate(tqdm(years)):
    vecs_y=vectors[vectors['year']==year]['vector']
    wiki_bench_y=wiki_bench[vectors['year']==year]['cos_sim']
    for i_c, csdg in enumerate(cSDGS_names):
        cos_sims_cSDGs[i_y*l_csdgs+i_c]['cSDG']=csdg
        cos_sims_cSDGs[i_y*l_csdgs+i_c]['all']=np.zeros(len(vecs_y))
        cos_sims_cSDGs[i_y*l_csdgs+i_c]['p_val']=np.zeros(len(vecs_y))
        for i_vec, vec in enumerate(vecs_y):
            _cos_sim=float(util.cos_sim(clustered_sdgs[i_c], vec))
            cos_sims_cSDGs[i_y*l_csdgs+i_c]['all'][i_vec]=_cos_sim
            cos_sims_cSDGs[i_y*l_csdgs+i_c]['p_val'][i_vec]=np.sum(wiki_bench_y[i_vec]>=_cos_sim)/len(wikivecs)
        cos_sims_cSDGs[i_y*l_csdgs+i_c]['mean']=np.mean(cos_sims_cSDGs[i_y*l_csdgs+i_c]['all'])

  0%|          | 0/9 [00:00<?, ?it/s]

In [80]:
for i_y, year in enumerate(tqdm(years)):
    print(f'\n{year:}')
    compa_y=vectors[vectors['year']==year]['company']
    sec_y=vectors[vectors['year']==year]['sector']
    for i_c, csdg in enumerate(cSDGS_names):
        _fdr_th=fdr_th(cos_sims_cSDGs[i_y*l_csdgs+i_c]['p_val'], alpha)
        validation_mask=cos_sims_cSDGs[i_y*l_csdgs+i_c]['p_val']<=_fdr_th
        validated_compa=compa_y[validation_mask]
        validated_sec=sec_y[validation_mask]
        if len(validated_compa)>0:
            print(f'\033[1m{csdg.capitalize():20}\033[0m len(validated_companies)={len(validated_compa):}')
            for i_vc, vc in enumerate(validated_compa):
                print(f'\t\033[1m{vc:25}\033[0m sector=\033[1m{sec_helper[validated_sec[i_vc]]:}\033[0m')
            print('\n')
        else:
            print(f'{csdg.capitalize():20} len(validated_companies)={len(validated_compa):}')

  0%|          | 0/9 [00:00<?, ?it/s]


2015
Social               len(validated_companies)=2
	unilever                  sector=Consumer Staples
	land_securities_group     sector=Real Estate


Economic             len(validated_companies)=1
	halma                     sector=Industrials


Environmental        len(validated_companies)=2
	halma                     sector=Industrials
	land_securities_group     sector=Real Estate



2016
Social               len(validated_companies)=2
	unilever                  sector=Consumer Staples
	standard_chartered        sector=Financials


Economic             len(validated_companies)=2
	standard_chartered        sector=Financials
	halma                     sector=Industrials


Environmental        len(validated_companies)=1
	halma                     sector=Industrials



2017
Social               len(validated_companies)=0
Economic             len(validated_companies)=1
	halma                     sector=Industrials


Environmental        len(validated_companies)=1
	halma                

## Each SDG

In [81]:
l_sdgs=len(sdg_helper.keys())

In [82]:
cos_sims_SDGs=np.zeros(len(years)*l_sdgs, dtype=[('SDG', 'U20'), ('mean', 'f8'), ('all', object), ('p_val', object)])

In [83]:
for i_y, year in enumerate(tqdm(years)):
    vecs_y=vectors[vectors['year']==year]['vector']
    wiki_bench_y=wiki_bench[vectors['year']==year]['cos_sim']
    for i_s, sdg in enumerate(sdg_helper.keys()):
        cos_sims_SDGs[i_y*l_sdgs+i_s]['SDG']=sdg
        cos_sims_SDGs[i_y*l_sdgs+i_s]['all']=np.zeros(len(vecs_y))
        cos_sims_SDGs[i_y*l_sdgs+i_s]['p_val']=np.zeros(len(vecs_y))
        for i_vec, vec in enumerate(vecs_y):
            _cos_sim=float(util.cos_sim(goalvecs[i_s], vec))
            cos_sims_SDGs[i_y*l_sdgs+i_s]['all'][i_vec]=_cos_sim
            cos_sims_SDGs[i_y*l_sdgs+i_s]['p_val'][i_vec]=np.sum(wiki_bench_y[i_vec]>=_cos_sim)/len(wikivecs)
        cos_sims_SDGs[i_y*l_sdgs+i_s]['mean']=np.mean(cos_sims_SDGs[i_y*l_sdgs+i_s]['all'])

  0%|          | 0/9 [00:00<?, ?it/s]

In [84]:
for i_y, year in enumerate(tqdm(years)):
    print(f'\n\033[1m{year:}\033[0m')
    compa_y=vectors[vectors['year']==year]['company']
    sec_y=vectors[vectors['year']==year]['sector']
    for i_s, sdg in enumerate(sdg_helper.keys()):
        _fdr_th=fdr_th(cos_sims_SDGs[i_y*l_sdgs+i_s]['p_val'], alpha)
        validation_mask=cos_sims_SDGs[i_y*l_sdgs+i_s]['p_val']<=_fdr_th
        validated_compa=compa_y[validation_mask]
        validated_sec=sec_y[validation_mask]
        if len(validated_compa)>0:
            print(f'\033[1m{sdg:}: {sdg_helper[sdg]:60}\033[0m len(validated_companies)={len(validated_compa):}')
            for i_vc, vc in enumerate(validated_compa):
                print(f'\t\033[1m{vc:25}\033[0m sector=\033[1m{sec_helper[validated_sec[i_vc]]:}\033[0m')
            print('\n')
        else:
            print(f'{sdg:}: {sdg_helper[sdg]:60} len(validated_companies)={len(validated_compa):}')

  0%|          | 0/9 [00:00<?, ?it/s]


2015
01: No Poverty                                                   len(validated_companies)=2
	unilever                  sector=Consumer Staples
	land_securities_group     sector=Real Estate


02: Zero hunger                                                  len(validated_companies)=0
03: Good health and well-being                                   len(validated_companies)=2
	gsk                       sector=Health Care
	land_securities_group     sector=Real Estate


04: Quality education                                            len(validated_companies)=0
05: Gender equality                                              len(validated_companies)=0
06: Clean water and sanitation                                   len(validated_companies)=3
	reckitt_benckiser_group   sector=Consumer Staples
	halma                     sector=Industrials
	land_securities_group     sector=Real Estate


07: Affordable and clean energy                                  len(validated_companies)=1
	halma      